In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import ast
from glob import glob
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

The performance of 3 models are analyzed under their respective header in this notebook (i.e., CheXpert-trained, MIMIC-trained, NIH-trained models)

# CheXpert v2

In [ ]:
import pandas as pd
import ast
from sklearn import metrics

def evaluate_race_csv(csv_file_path, label_name):
    try:
        predict_df = pd.read_csv(csv_file_path)

        race_map = {
            0: 'Hispanic',
            1: 'Asian',
            2: 'Black',
            3: 'White'
        }

        print(f"\n=== {label_name} ===")
        value_counts = predict_df['Race/Ethnicity_Tensor_Id'].value_counts(normalize=True).sort_index()

        for race_id, proportion in value_counts.items():
            race_name = race_map.get(race_id, f"Unknown ({race_id})")
            print(f"{race_name}: {proportion * 100:.2f}% of samples")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")


# Define files and labels
csv_files_with_labels = [
    ('/content/chexpert_race_v2_brax_test.csv', 'BRAX'),
    ('/content/chexpert_race_v2_india_test.csv', 'India TB'),
    ('/content/chexpert_race2_jsrt_test.csv', 'JSRT'),
    ('/content/chexpert_race2_padchest_test.csv', 'PadChest'),
    ('/content/chexpert_race2_vindr_test.csv', 'VinDr'),
    ('/content/chexpert_race2_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/chexpert_race2_shenzhen_test.csv', 'Shenzhen')
]

# Run evaluations
for file_path, label in csv_files_with_labels:
    evaluate_race_csv(file_path, label)



=== BRAX ===
Hispanic: 12.67% of samples
Asian: 9.57% of samples
Black: 7.20% of samples
White: 70.56% of samples

=== India TB ===
Hispanic: 12.16% of samples
Asian: 30.63% of samples
Black: 16.67% of samples
White: 40.54% of samples

=== JSRT ===
Asian: 3.64% of samples
White: 96.36% of samples


<ipython-input-36-80f349d601d8>:7: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  predict_df = pd.read_csv(csv_file_path)



=== PadChest ===
Black: 15.38% of samples
White: 84.62% of samples

=== VinDr ===
Hispanic: 3.77% of samples
Asian: 85.48% of samples
Black: 3.04% of samples
White: 7.72% of samples

=== VinDr-PCXR ===
Hispanic: 23.05% of samples
Asian: 10.85% of samples
Black: 63.82% of samples
White: 2.28% of samples

=== Shenzhen ===
Asian: 26.28% of samples
White: 73.72% of samples


In [ ]:
import pandas as pd
import numpy as np

def bootstrap_race_prevalence(csv_file_path, label_name, n_iterations=1000, random_state=2025):
    try:
        df = pd.read_csv(csv_file_path)

        race_map = {
            1: 'Asian',
            2: 'Black',
            3: 'White',
            0: 'Hispanic'
        }

        df = df[df['Race/Ethnicity_Tensor_Id'].isin(race_map.keys())].copy()
        y = df['Race/Ethnicity_Tensor_Id'].values
        rng = np.random.default_rng(seed=random_state)

        prevalence_bootstrap = {k: [] for k in race_map.keys()}

        for _ in range(n_iterations):
            sample = rng.choice(y, size=len(y), replace=True)
            counts = np.bincount(sample, minlength=4) / len(sample)
            for i in race_map.keys():
                prevalence_bootstrap[i].append(counts[i])

        print(f"\n{label_name} - Bootstrapped Race Prevalence (95% CI):")
        for race_id, samples in prevalence_bootstrap.items():
            race_name = race_map[race_id]
            ci_low, ci_high = np.percentile(samples, [2.5, 97.5])
            print(f"{race_name:<10}: 95% CI: [{ci_low*100:.2f}, {ci_high*100:.2f}]")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")

# Define files and labels
csv_files_with_labels = [
    ('/content/chexpert_race_v2_brax_test.csv', 'BRAX'),
    ('/content/chexpert_race_v2_india_test.csv', 'India TB'),
    ('/content/chexpert_race2_jsrt_test.csv', 'JSRT'),
    ('/content/chexpert_race2_padchest_test.csv', 'PadChest'),
    ('/content/chexpert_race2_vindr_test.csv', 'VinDr'),
    ('/content/chexpert_race2_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/chexpert_race2_shenzhen_test.csv', 'Shenzhen')
]

# Run evaluations
for file_path, label in csv_files_with_labels:
    bootstrap_race_prevalence(file_path, label)


BRAX - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [9.17, 9.95]
Black     : 95% CI: [6.84, 7.59]
White     : 95% CI: [69.90, 71.22]
Hispanic  : 95% CI: [12.19, 13.15]

India TB - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [24.77, 36.50]
Black     : 95% CI: [12.16, 21.62]
White     : 95% CI: [34.23, 46.85]
Hispanic  : 95% CI: [7.66, 16.67]

JSRT - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [1.62, 6.07]
Black     : 95% CI: [0.00, 0.00]
White     : 95% CI: [93.93, 98.38]
Hispanic  : 95% CI: [0.00, 0.00]


<ipython-input-37-daf71081d43c>:6: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)



PadChest - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 0.00]
Black     : 95% CI: [15.14, 15.59]
White     : 95% CI: [84.41, 84.86]
Hispanic  : 95% CI: [0.00, 0.00]

VinDr - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [84.97, 86.02]
Black     : 95% CI: [2.82, 3.28]
White     : 95% CI: [7.30, 8.13]
Hispanic  : 95% CI: [3.47, 4.04]

VinDr-PCXR - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [10.20, 11.48]
Black     : 95% CI: [62.82, 64.76]
White     : 95% CI: [1.98, 2.59]
Hispanic  : 95% CI: [22.20, 23.85]

Shenzhen - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [22.96, 29.46]
Black     : 95% CI: [0.00, 0.00]
White     : 95% CI: [70.54, 77.04]
Hispanic  : 95% CI: [0.00, 0.00]


# MIMIC

In [ ]:
import pandas as pd
import ast
from sklearn import metrics

def evaluate_race_csv(csv_file_path, label_name):
    try:
        predict_df = pd.read_csv(csv_file_path)

        race_map = {
            0: 'Asian',
            1: 'Black',
            3: 'Other',
            4: 'White',
            2: 'Hispanic',
        }

        print(f"\n=== {label_name} ===")
        value_counts = predict_df['Race/Ethnicity_Tensor_Id'].value_counts(normalize=True).sort_index()

        for race_id, proportion in value_counts.items():
            race_name = race_map.get(race_id, f"Unknown ({race_id})")
            print(f"{race_name}: {proportion * 100:.2f}% of samples")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")


# Define files and labels
csv_files_with_labels = [
    ('/content/mimic_race_brax_test.csv', 'BRAX'),
    ('/content/mimic_race_india_test.csv', 'India TB'),
    ('/content/mimic_race_jsrt_test.csv', 'JSRT'),
    ('/content/mimic_race_padchest_test.csv', 'PadChest'),
    ('/content/mimic_race_vindr_test.csv', 'VinDr'),
    ('/content/mimic_race_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/mimic_race_shenzhen_test.csv', 'Shenzhen')
]

# Run evaluations
for file_path, label in csv_files_with_labels:
    evaluate_race_csv(file_path, label)


=== BRAX ===
Asian: 4.65% of samples
Black: 13.07% of samples
Hispanic: 2.02% of samples
White: 80.26% of samples

=== India TB ===
Asian: 15.32% of samples
Black: 59.01% of samples
White: 25.68% of samples

=== JSRT ===
White: 100.00% of samples


<ipython-input-38-1daf50b830dc>:7: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  predict_df = pd.read_csv(csv_file_path)



=== PadChest ===
White: 100.00% of samples

=== VinDr ===
Asian: 73.37% of samples
Black: 14.57% of samples
Hispanic: 0.79% of samples
White: 11.27% of samples

=== VinDr-PCXR ===
Asian: 17.44% of samples
Black: 73.19% of samples
Hispanic: 0.10% of samples
White: 9.27% of samples

=== Shenzhen ===
White: 100.00% of samples


In [ ]:
import pandas as pd
import numpy as np

def bootstrap_race_prevalence(csv_file_path, label_name, n_iterations=1000, random_state=2025):
    try:
        df = pd.read_csv(csv_file_path)

        race_map = {
            0: 'Asian',
            1: 'Black',
            3: 'Other',
            4: 'White',
            2: 'Hispanic',
        }

        df = df[df['Race/Ethnicity_Tensor_Id'].isin(race_map.keys())].copy()
        y = df['Race/Ethnicity_Tensor_Id'].values
        rng = np.random.default_rng(seed=random_state)

        prevalence_bootstrap = {k: [] for k in race_map.keys()}

        for _ in range(n_iterations):
            sample = rng.choice(y, size=len(y), replace=True)
            counts = np.bincount(sample, minlength=4) / len(sample)
            for i in race_map.keys():
                prevalence_bootstrap[i].append(counts[i])

        print(f"\n{label_name} - Bootstrapped Race Prevalence (95% CI):")
        for race_id, samples in prevalence_bootstrap.items():
            race_name = race_map[race_id]
            ci_low, ci_high = np.percentile(samples, [2.5, 97.5])
            print(f"{race_name:<10}: 95% CI: [{ci_low*100:.2f}, {ci_high*100:.2f}]")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")

# Define files and labels
csv_files_with_labels = [
    ('/content/mimic_race_brax_test.csv', 'BRAX'),
    ('/content/mimic_race_india_test.csv', 'India TB'),
    ('/content/mimic_race_jsrt_test.csv', 'JSRT'),
    ('/content/mimic_race_padchest_test.csv', 'PadChest'),
    ('/content/mimic_race_vindr_test.csv', 'VinDr'),
    ('/content/mimic_race_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/mimic_race_shenzhen_test.csv', 'Shenzhen')
]


# Run evaluations
for file_path, label in csv_files_with_labels:
    bootstrap_race_prevalence(file_path, label)


BRAX - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [4.35, 4.95]
Black     : 95% CI: [12.54, 13.52]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [79.73, 80.86]
Hispanic  : 95% CI: [1.82, 2.23]

India TB - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [10.81, 19.82]
Black     : 95% CI: [53.14, 65.77]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [19.82, 31.08]
Hispanic  : 95% CI: [0.00, 0.00]

JSRT - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 0.00]
Black     : 95% CI: [0.00, 0.00]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [100.00, 100.00]
Hispanic  : 95% CI: [0.00, 0.00]


<ipython-input-39-db70f3cb03fd>:6: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)



PadChest - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 0.00]
Black     : 95% CI: [0.00, 0.00]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [100.00, 100.00]
Hispanic  : 95% CI: [0.00, 0.00]

VinDr - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [72.73, 74.04]
Black     : 95% CI: [14.08, 15.12]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [10.79, 11.69]
Hispanic  : 95% CI: [0.67, 0.92]

VinDr-PCXR - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [16.62, 18.19]
Black     : 95% CI: [72.31, 74.13]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [8.70, 9.87]
Hispanic  : 95% CI: [0.04, 0.16]

Shenzhen - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 0.00]
Black     : 95% CI: [0.00, 0.00]
Other     : 95% CI: [0.00, 0.00]
White     : 95% CI: [100.00, 100.00]
Hispanic  : 95% CI: [0.00, 0.00]


# We also trained a classifier on CheXpert where only non-Hispanic Asian, non-Hispanic Black and non-Hispanic White individuals were included. These are the results. CheXpert v1

In [ ]:
import pandas as pd
import ast
from sklearn import metrics

def evaluate_race_csv(csv_file_path, label_name):
    try:
        predict_df = pd.read_csv(csv_file_path)

        race_map = {
            0: 'Asian',
            1: 'Black',
            3: 'White',
            2: 'Hispanic'
        }

        print(f"\n=== {label_name} ===")
        value_counts = predict_df['Race/Ethnicity_Tensor_Id'].value_counts(normalize=True).sort_index()

        for race_id, proportion in value_counts.items():
            race_name = race_map.get(race_id, f"Unknown ({race_id})")
            print(f"{race_name}: {proportion * 100:.2f}% of samples")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")


# Define files and labels
csv_files_with_labels = [
    ('/content/chexpert_race_v1_brax_test.csv', 'BRAX'),
    ('/content/chexpert_race_v1_india_test.csv', 'India TB'),
    ('/content/chexpert_race1_jsrt_test.csv', 'JSRT'),
    ('/content/chexpert_race1_padchest_test.csv', 'PadChest'),
    ('/content/chexpert_race1_vindr_test.csv', 'VinDr'),
    ('/content/chexpert_race1_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/chexpert_race1_shenzhen_test.csv', 'Shenzhen')
]

# Run evaluations
for file_path, label in csv_files_with_labels:
    evaluate_race_csv(file_path, label)



=== BRAX ===
Asian: 12.29% of samples
Black: 14.87% of samples
Hispanic: 72.83% of samples

=== India TB ===
Asian: 40.54% of samples
Black: 38.29% of samples
Hispanic: 21.17% of samples

=== JSRT ===
Asian: 0.40% of samples
Hispanic: 99.60% of samples

=== PadChest ===
Black: 0.00% of samples
Hispanic: 100.00% of samples

=== VinDr ===
Asian: 86.13% of samples
Black: 6.44% of samples
Hispanic: 7.43% of samples
Error processing /content/chexpert_race1_vindr_peds_test.csv: [Errno 2] No such file or directory: '/content/chexpert_race1_vindr_peds_test.csv'

=== Shenzhen ===
Asian: 0.45% of samples
Black: 1.66% of samples
Hispanic: 97.89% of samples


<ipython-input-40-e5fa89e9edf2>:7: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  predict_df = pd.read_csv(csv_file_path)


In [ ]:
import pandas as pd
import numpy as np

def bootstrap_race_prevalence(csv_file_path, label_name, n_iterations=1000, random_state=2025):
    try:
        df = pd.read_csv(csv_file_path)

        race_map = {
            0: 'Asian',
            1: 'Black',
            3: 'White',
            2: 'Hispanic'
        }

        df = df[df['Race/Ethnicity_Tensor_Id'].isin(race_map.keys())].copy()
        y = df['Race/Ethnicity_Tensor_Id'].values
        rng = np.random.default_rng(seed=random_state)

        prevalence_bootstrap = {k: [] for k in race_map.keys()}

        for _ in range(n_iterations):
            sample = rng.choice(y, size=len(y), replace=True)
            counts = np.bincount(sample, minlength=4) / len(sample)
            for i in race_map.keys():
                prevalence_bootstrap[i].append(counts[i])

        print(f"\n{label_name} - Bootstrapped Race Prevalence (95% CI):")
        for race_id, samples in prevalence_bootstrap.items():
            race_name = race_map[race_id]
            ci_low, ci_high = np.percentile(samples, [2.5, 97.5])
            print(f"{race_name:<10}: 95% CI: [{ci_low*100:.2f}, {ci_high*100:.2f}]")

    except (FileNotFoundError, KeyError, ValueError) as e:
        print(f"Error processing {csv_file_path}: {e}")

# Define files and labels
csv_files_with_labels = [
    ('/content/chexpert_race_v1_brax_test.csv', 'BRAX'),
    ('/content/chexpert_race_v1_india_test.csv', 'India TB'),
    ('/content/chexpert_race1_jsrt_test.csv', 'JSRT'),
    ('/content/chexpert_race1_padchest_test.csv', 'PadChest'),
    ('/content/chexpert_race1_vindr_test.csv', 'VinDr'),
    ('/content/chexpert_race1_vindr_peds_test.csv', 'VinDr-PCXR'),
    ('/content/chexpert_race1_shenzhen_test.csv', 'Shenzhen')
]

# Run evaluations
for file_path, label in csv_files_with_labels:
    bootstrap_race_prevalence(file_path, label)


BRAX - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [11.84, 12.74]
Black     : 95% CI: [14.36, 15.40]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [72.22, 73.47]

India TB - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [34.23, 46.85]
Black     : 95% CI: [31.98, 45.05]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [16.22, 26.14]

JSRT - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 1.21]
Black     : 95% CI: [0.00, 0.00]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [98.79, 100.00]


<ipython-input-41-b279a3eda629>:6: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)



PadChest - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 0.00]
Black     : 95% CI: [0.00, 0.01]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [99.99, 100.00]

VinDr - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [85.59, 86.65]
Black     : 95% CI: [6.07, 6.81]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [7.05, 7.83]
Error processing /content/chexpert_race1_vindr_peds_test.csv: [Errno 2] No such file or directory: '/content/chexpert_race1_vindr_peds_test.csv'

Shenzhen - Bootstrapped Race Prevalence (95% CI):
Asian     : 95% CI: [0.00, 1.06]
Black     : 95% CI: [0.76, 2.57]
White     : 95% CI: [0.00, 0.00]
Hispanic  : 95% CI: [96.83, 98.94]
